# Amazon Rekognition Face API

This notebook contains example snippets for interacting with Face API.

First, confirm that the `boto3` module is installed.

In [5]:
!pip3 install boto3

You should consider upgrading via the 'c:\tools\python3.8\python.exe -m pip install --upgrade pip' command.


Configure the global settings for running this notebook.

1. Set the `bucket_name` to an Amazon S3 bucket within your AWS account.
1. Set the `region_name` to the AWS Region containing your bucket
1. Create the `rekognition` client for the same region as the bucket

In [4]:
import boto3

bucket_name = 'ch04-hotel-use2'
region_name = 'us-east-2'
image_name = 'images/Nate-Bachmeier.png'
rekognition = boto3.client('rekognition', region_name=region_name)

Invoke the `DetectFaces` API 

In [5]:
faces = rekognition.detect_faces(
    Attributes=['ALL'],
    Image={
     'S3Object':{
        'Bucket': bucket_name,
        'Name': image_name
      }
    })

Next, let's create the `print_faces` function for pretty printing the `faces` response.

In [11]:
def print_faces(faces):
    counter=0
    for face in faces['FaceDetails']:
        counter+=1
        age_range = face['AgeRange']
        gender = face['Gender']
        emotions = face['Emotions']
        
        age = (age_range['High'] + age_range['Low'])/2
        emotions.sort(key=lambda x: x['Confidence'], reverse=True)
        top_emotion = emotions[0]

        print('Person %d is %s(%2.2f%%) around %d age and %s(%2.2f%%) state.' % (
            counter,
            gender['Value'],
            gender['Confidence'],
            age,
            top_emotion['Type'],
            top_emotion['Confidence']
        ))

print_faces(faces)

Person 1 is Male(100.00%) around 31 age and CALM(96.24%) state.


You can build policies that `check_faces` is valid and meets our needs.

In [16]:
def has_only_one_face(faces):
    if len(faces['FaceDetails']) == 1:
       return True
    return False

def is_facing_forward(face):
    for dimension in ['Pitch','Roll','Yaw']:
        value = face['Pose'][dimension]
        if not (-45 < value and value < 45):
            return False
    return True

def has_sunglasses(face):
    sunglasses = face['Sunglasses']['Value']
    return sunglasses

def is_well_lit(face):
    if face['Quality']['Brightness'] < 25:
        return False    
    return True

def check_faces(faces):
    if not has_only_one_face(faces):
        print('Incorrect face count.')
        return False

    user_face = faces['FaceDetails'][0]
    if not is_facing_forward(user_face):
        print('Customer not facing forward')
        return False

    if has_sunglasses(user_face):
        print('Please take off sunglasses.')
        return False

    if not is_well_lit(user_face):
        print('The image is blurry')
        return False

    print("Valid face detected!")
    return True

check_faces(faces)

Valid face detected!


True

Next, let's use the `Index` API to persist FaceMetadata into the `HotelCollection`.

In [22]:
collection_id="HotelCollection"
image_set = {
    'nbachmei': 'images/Nate-Bachmeier.png',
    'lemull': 'images/Lauren-Mullennex.jpg'
}

for (externalImageId, object_name) in image_set.items():
    rekognition.index_faces(
        CollectionId = collection_id,
        ExternalImageId=externalImageId,
        Image={
            'S3Object':{
                'Bucket': bucket_name,
                'Name': object_name
            }
        })

Use the `SearchFacesByImage` API to find the indexed face using a different image of that person.

In [26]:
search_image = 'images/SearchFacesByImageExample.jpg'

response = rekognition.search_faces_by_image(
    CollectionId=collection_id,
    Image={
        'S3Object':{
            'Bucket': bucket_name,
            'Name': search_image
        }
    })

Let's pretty print the response and confirm the `ExternalImageId` is Nate...

In [28]:
def print_search_results(search_response):
    for match in search_response['FaceMatches']:
        externalImageId = match['Face']['ExternalImageId']
        confidence = match['Face']['Confidence']

        print('This image is %s (%2.2f%% confidence)' % (
            externalImageId,
            confidence
        ))

print_search_results(response)

This image is nbachmei (100.00% confidence)


This snippet creates the Amazon Textract client and invokes the `AnalyzeId` API. You can use this action to parse government identity cards like drivers licensing and passport cards. 

In [32]:
textract = boto3.client('textract', region_name=region_name)
passport_card = 'images/passport_card.jpeg'

response = textract.analyze_id(
    DocumentPages=[
        {
            "S3Object":{
                "Bucket": bucket_name,
                "Name": passport_card
            }
        }
    ])


Let's pretty print the `response` from the `AnalyzeId` API.

In [35]:
from json import dumps

fields = {}
for document in response['IdentityDocuments']:    
    for field in document['IdentityDocumentFields']:
        key = field['Type']['Text']
        value = field['ValueDetection']['Text']    
        fields[key] = value

print(dumps(fields,indent=2))

{
  "FIRST_NAME": "HAPPY",
  "LAST_NAME": "TRAVELER",
  "MIDDLE_NAME": "",
  "SUFFIX": "",
  "CITY_IN_ADDRESS": "",
  "ZIP_CODE_IN_ADDRESS": "",
  "STATE_IN_ADDRESS": "",
  "STATE_NAME": "",
  "DOCUMENT_NUMBER": "C03005988",
  "EXPIRATION_DATE": "29 NOV 2019",
  "DATE_OF_BIRTH": "1 JAN 1981",
  "DATE_OF_ISSUE": "30 NOV 2009",
  "ID_TYPE": "DRIVER LICENSE FRONT",
  "ENDORSEMENTS": "",
  "VETERAN": "",
  "RESTRICTIONS": "",
  "CLASS": "",
  "ADDRESS": "",
  "COUNTY": "",
  "PLACE_OF_BIRTH": "NEW YORK U.S.A.",
  "MRZ_CODE": ""
}


Use the `CompareFaces` to determine the similar between two images.

In [45]:
nbachmei_image = "images/Nate-Bachmeier.png"
match_passport = "images/nbachmei-passport.jpg"
nonmatch_passport = "images/passport_card.jpeg"

def compare_faces(passport_card):
    comparison_match = rekognition.compare_faces(
        SourceImage={
            "S3Object":{
                "Bucket": bucket_name,
                "Name": nbachmei_image
            }
        },
        TargetImage={
            "S3Object":{
                "Bucket": bucket_name,
                "Name": passport_card
            }
        })

    if len(comparison_match['FaceMatches'])  > 0:
        for face in comparison_match['FaceMatches']:
            similarity = face['Similarity']
            print('%s face with %2.2f%% similarity ' % (
                passport_card, 
                similarity))
    else:
        print("%s has %d unmatched faces" % (
            passport_card,
            len(comparison_match['UnmatchedFaces']))
        )

compare_faces(match_passport)
compare_faces(nonmatch_passport)

images/nbachmei-passport.jpg face with 98.77% similarity 
images/passport_card.jpeg has 1 unmatched faces
